In [1]:
import fitz
import re
import pandas

In [2]:
pdf_document = "/Users/a/Desktop/Book of isms.pdf"

In [3]:
doc = fitz.open(pdf_document)
print("Pages: %i" % doc.pageCount)
print(doc.metadata)

Pages: 251
{'format': 'PDF 1.6', 'title': None, 'author': None, 'subject': None, 'keywords': None, 'creator': 'Adobe InDesign CS4 (6.0.4)', 'producer': 'macOS 版本10.15（版号19A583） Quartz PDFContext, AppendMode 1.1', 'creationDate': "D:20100421122035+01'00'", 'modDate': "D:20191010052259Z00'00'", 'encryption': None}


In [4]:
page = doc.loadPage(3)
indexes = page.getTextBlocks()[2][4]
index_list = re.findall(r"\d+\.?\d*", indexes)
print(index_list)

real_index_list = [int(i)+6 for i in index_list]
print(real_index_list)

['1', '24', '35', '52', '61', '72', '81', '86', '93', '99', '105', '109', '117', '142', '155', '160', '185', '187', '196', '215', '223', '228', '231', '234']
[7, 30, 41, 58, 67, 78, 87, 92, 99, 105, 111, 115, 123, 148, 161, 166, 191, 193, 202, 221, 229, 234, 237, 240]


### 7-251

In [5]:
ismsDF = pandas.DataFrame(columns=["id", "word", "explain"])

In [6]:
all_isms = []
temp = ""
word = ""
explain = ""
ID = 0
all_isms = []
data = {}

for i in range(7, 251):
    page = doc.loadPage(i)
    if i not in real_index_list:
        contents = page.getTextBlocks()[1:-1]
    else:
        contents = page.getTextBlocks()[:-1]
        
    for content in contents:
        ism = content[4].replace("\n", "")
        if len(ism) != 1:
            if not ism.endswith("."):
                temp = temp + ism
            elif temp != "":
                temp += ism
                j = 1
                while not temp[:j].endswith("ism"):
                    j += 1
                
                word = temp[:j]
                explain = temp[j+1:].capitalize()
                ID += 1
                temp = ""
                
                data = {
                    'id' : ID,
                    'word' : word,
                    'explain' : explain
                }
                ismsDF = ismsDF.append(data, ignore_index=True)
                
            else:
                j = 1
                while not ism[:j].endswith("ism"):
                    j += 1
                
                word = ism[:j]
                explain = ism[j+1:].capitalize()
                ID += 1
                
                data = {
                    'id' : ID,
                    'word' : word,
                    'explain' : explain
                }
                ismsDF = ismsDF.append(data, ignore_index=True)

In [7]:
ismsDF.head()

,id,word,explain
0,1,Abolitionism,A movement in europe and the americas dedicate...
1,2,Absenteeism,The state of being absent from the workplace –...
2,3,Absolutism,"As a political doctrine, it asserts the unlimi..."
3,4,Abstract expressionism,A school of non-figurative art that began in n...
4,5,Absurdism,"A philosophy, closely linked to existentialism..."


## 导出DataFrame

In [8]:
ismsDF.to_csv("Isms.csv")

In [9]:
ismsDF.to_json("Isms.js")
ismsDF.to_json("Isms2.js", orient="records")

In [10]:
# s = "Abstract Alarmism An addiction, sometimes associated with mental illness, to alcoholic drinks, which an alcoholic consumes to excess. America’s national council on Alcoholism and Drug Dependence defines alcoholism as “a primary, chronic disease characterised by impaired control over drinking, preoccupation with the drug alcohol, use of alcohol despite adverse consequences, and distortions in thinking”. the description of alcoholism as a disease, rather than simply a form of behaviour, became pr"
# re.findall(r'\S*?ism\b', s)

## 导出主义列表

In [11]:
import json

In [12]:
ismsDF['word'] = [s.capitalize() for s in ismsDF.word.to_list()]

In [13]:
ismsDF['alphabet'] = ismsDF.word.str[0]

In [17]:
words_dic = {}
for item in ismsDF.groupby(by=['alphabet']):
    dic = {}
    for i in range(0, item[1].shape[0]):
        word = item[1].word.tolist()[i]
        explain = item[1].explain.tolist()[i]
        dic[word] = explain
        
    words_dic[item[0]] = dic

In [19]:
with open('explains.js', 'w') as f:
    json.dump(words_dic, f)

In [16]:
# words_dic